In [1]:
import os
import json
import pandas as pd 
import traceback


In [2]:
from langchain.chat_models import ChatOpenAI

In [5]:
from dotenv import load_dotenv

load_dotenv()

KEY = os.getenv("OPENAI_API_KEY")

In [7]:
llm = ChatOpenAI(openai_api_key = KEY,
                 model_name ='gpt-3.5-turbo',
                 temperature=0.5)

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [19]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quize of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [20]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [21]:
quiz_chain = LLMChain(llm=llm,
                      prompt=quiz_generation_prompt,
                      output_key="quiz",
                      verbose=True)

In [22]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [23]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [24]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key="review",
    verbose=True
)

In [25]:
generator_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [26]:
file_path = r"D:\project\QnA_Generator\data.txt"

In [27]:
with open(file_path, 'r') as file:
    TEXT = file.read()
    

In [28]:
print(TEXT)


Shantanu, the king of Hastinapur, was married to Ganga (personification of the Ganges) with whom he had a son called Devavrat. Several years later, 
when Devavrat had grown up to be an accomplished prince, Shantanu fell in love with Satyavati. Her father refused to let her marry the king unless 
the king promised that Satyavati's son and descendants would inherit the throne. Unwilling to deny Devavrat his rights, Shantanu declined to do so but
the prince, on coming to know of the matter, rode over to Satyavati's house, vowed to renounce the throne and to remain celibate throughout his 
life. The prince then took Satyavati home to the palace so that the king, his father, could marry her. On account of the terrible vow that he'd 
taken that day, Devavrat came to be known as Bheeshm. Shantanu was so pleased with his son that he granted to Devavrat the boon of choosing the 
time of his own death.

In time, Shantanu and Satyavati had two sons. Soon thereafter, Shantanu died. Satyavati's so

In [29]:
#serlialize the python dictonary into a JSON formated string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [32]:
NUMBER = 5
SUBJECT = "MAHABHARAT"
TONE = "simple"

In [33]:
#how to setup Token Usage Tracking in Langchain
with get_openai_callback() as cb:
    response = generator_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject" : SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
Shantanu, the king of Hastinapur, was married to Ganga (personification of the Ganges) with whom he had a son called Devavrat. Several years later, 
when Devavrat had grown up to be an accomplished prince, Shantanu fell in love with Satyavati. Her father refused to let her marry the king unless 
the king promised that Satyavati's son and descendants would inherit the throne. Unwilling to deny Devavrat his rights, Shantanu declined to do so but
the prince, on coming to know of the matter, rode over to Satyavati's house, vowed to renounce the throne and to remain celibate throughout his 
life. The prince then took Satyavati home to the palace so that the king, his father, could marry her. On account of the terrible vow that he'd 
taken that day, Devavrat came to be known as Bheeshm. Shantanu was so pleased with his son that he granted to Devavrat the boon of choosing the 
time of 

In [34]:
print(f"total tokens:{cb.total_tokens}")
print(f"Prompt tokens:{cb.prompt_tokens}")
print(f"completion tokens:{cb.completion_tokens}")
print(f"total cost:{cb.total_cost}")

total tokens:1513
Prompt tokens:1123
completion tokens:390
total cost:0.0024644999999999997


In [35]:
response

{'text': "\nShantanu, the king of Hastinapur, was married to Ganga (personification of the Ganges) with whom he had a son called Devavrat. Several years later, \nwhen Devavrat had grown up to be an accomplished prince, Shantanu fell in love with Satyavati. Her father refused to let her marry the king unless \nthe king promised that Satyavati's son and descendants would inherit the throne. Unwilling to deny Devavrat his rights, Shantanu declined to do so but\nthe prince, on coming to know of the matter, rode over to Satyavati's house, vowed to renounce the throne and to remain celibate throughout his \nlife. The prince then took Satyavati home to the palace so that the king, his father, could marry her. On account of the terrible vow that he'd \ntaken that day, Devavrat came to be known as Bheeshm. Shantanu was so pleased with his son that he granted to Devavrat the boon of choosing the \ntime of his own death.\n\nIn time, Shantanu and Satyavati had two sons. Soon thereafter, Shantanu d

In [37]:
quiz = response.get("quiz")

In [40]:
quiz = json.loads(quiz)

In [41]:
#creating Quiz from the above data
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [42]:
quiz_table_data

[{'MCQ': 'Who was Shantanu married to?',
  'Choices': 'a: Ganga | b: Satyavati | c: Devavrat | d: Vichitravirya',
  'Correct': 'a'},
 {'MCQ': "Why did Shantanu decline to promise Satyavati's father?",
  'Choices': "a: He didn't love Satyavati | b: He didn't want Devavrat to inherit the throne | c: He wanted to marry someone else | d: He wanted to remain celibate",
  'Correct': 'b'},
 {'MCQ': "What was Devavrat's vow?",
  'Choices': 'a: To renounce the throne | b: To remain celibate | c: Both a and b | d: None of the above',
  'Correct': 'c'},
 {'MCQ': "Who did Shantanu marry after Devavrat's vow?",
  'Choices': 'a: Ganga | b: Satyavati | c: Devavrat | d: Vichitravirya',
  'Correct': 'b'},
 {'MCQ': 'How did Devavrat come to be known as Bheeshm?',
  'Choices': 'a: By renouncing the throne | b: By remaining celibate | c: By marrying Satyavati | d: By granting his own death',
  'Correct': 'a'}]

In [44]:
quiz_df = pd.DataFrame(quiz_table_data)

In [45]:
quiz_df.to_csv("Mahabharat_quiz.csv", index=False)